In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_ta as ta
import re
import requests

from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
from invaas.schwab.schwab_task import SchwabTask

task = SchwabTask(env="local")
await task.setup_api()

2024-05-11 23:32:06 WARNING No DBUtils defined in the runtime
2024-05-11 23:32:06 INFO Initializing task for local environment
2024-05-11 23:32:06 INFO Logging in to Schwab
2024-05-11 23:32:43 INFO Net worth: 1130.48
2024-05-11 23:32:44 INFO Available cash to buy stocks: 1130.48
2024-05-11 23:32:45 INFO Available cash to buy options: 1130.48


In [3]:
def get_options_chains_data(ticker: str):
    options_info = requests.get(f"https://client.schwab.com/symlup/OptionsCwp/Underlying/{ticker}.txt").text
    options_info = re.sub(r"[\n\t\r]*", "", options_info)
    options_info = json.loads(options_info.replace('SuggestionBox.JsonpCallback("option",', "")[:-1])
    expirations = [x["Date"] for x in options_info["Expirations"]]
    options_chain_url = f"https://ausgateway.schwab.com/api/is.CSOptionChainsWeb/v1/OptionChainsPort/OptionChains/chains?Symbol={ticker}&IncludeGreeks=false&{','.join(expirations)}"
    task.schwab_api.update_token("update")
    return json.loads(requests.get(options_chain_url, headers=task.schwab_api.headers).text)


options_chains_data = get_options_chains_data("QQQ")
options_chains_data

{'UnderlyingData': {'Ask': '441.25',
  'AskSize': '1',
  'Bid': '441.22',
  'BidSize': '1',
  'Close': '441.02',
  'CumulativeVolume': '27116566',
  'Description': 'INVSC QQQ TRUST SRS 1',
  'ErrorCode': 0,
  'InstrumentType': 'ETF',
  'Last': '442.06',
  'NetChange': '1.04',
  'PercentChange': '0.24',
  'Open': '442.54',
  'DailyHigh': '444.31',
  'DailyLow': '440.499',
  'High52W': '449.34',
  'Low52W': '322.94',
  'Symbol': 'QQQ',
  'DividendYield': 0.52,
  'ExchangeName': 'NASDAQ',
  'IsDelayed': False},
 'Expirations': [{'ExpirationGroup': {'RootSymbol': 'QQQ',
    'MonthAndDay': 'May 13',
    'Year': '2024',
    'Day': 'Mon',
    'SettlementType': '',
    'DaysUntil': 2},
   'AdjustedLabel': {},
   'AdjustedGroup': '',
   'Chains': [{'SymbolGroup': 'QQQ20240513359',
     'Legs': [{'Sym': 'QQQ   240513C00359000',
       'OptionType': 'C',
       'Strk': '359',
       'Chg': '0',
       'ChgPct': '0',
       'Bid': '82.77',
       'Ask': '83.16',
       'Lst': '0',
       'Vol': '0

In [4]:
data = []
underlying_last = float(options_chains_data["UnderlyingData"]["Last"])

for expiration in options_chains_data["Expirations"]:
    days_until_expiration = expiration["ExpirationGroup"]["DaysUntil"]
    expiration_month_day = expiration["ExpirationGroup"]["MonthAndDay"].split(" ")
    expiration_month = expiration_month_day[0]
    expiration_day = expiration_month_day[1]
    expiration_year = expiration["ExpirationGroup"]["Year"]
    expiration_date = datetime.strptime(f"{expiration_month} {expiration_day} {expiration_year}", "%b %d %Y")

    for chain in expiration["Chains"]:
        data_obj = {}
        data_obj["OPTION_ID"] = chain["SymbolGroup"]
        data_obj["UNDERLYING_LAST"] = underlying_last
        data_obj["EXPIRE_DATE"] = expiration_date
        data_obj["DTE"] = days_until_expiration

        for leg in chain["Legs"]:
            data_obj["STRIKE"] = float(leg["Strk"])
            data_obj["STRIKE_DISTANCE"] = abs(data_obj["STRIKE"] - underlying_last)
            data_obj["STRIKE_DISTANCE_PCT"] = data_obj["STRIKE_DISTANCE"] / underlying_last
            leg_prefix = leg["OptionType"] + "_"
            data_obj[leg_prefix + "ID"] = leg["Sym"]
            data_obj[leg_prefix + "BID"] = float(leg["Bid"])
            data_obj[leg_prefix + "ASK"] = float(leg["Ask"])
            data_obj[leg_prefix + "VOLUME"] = int(leg["Vol"])

        data.append(data_obj)

df_options_chains = pd.DataFrame(data=data)
df_options_chains

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
0,QQQ20240513359,442.06,2024-05-13,2,359.0,83.06,0.187893,QQQ 240513C00359000,82.77,83.16,0,QQQ 240513P00359000,0.00,0.01,26
1,QQQ20240513360,442.06,2024-05-13,2,360.0,82.06,0.185631,QQQ 240513C00360000,81.77,82.17,4,QQQ 240513P00360000,0.00,0.01,0
2,QQQ20240513361,442.06,2024-05-13,2,361.0,81.06,0.183369,QQQ 240513C00361000,80.77,81.17,0,QQQ 240513P00361000,0.00,0.01,0
3,QQQ20240513362,442.06,2024-05-13,2,362.0,80.06,0.181107,QQQ 240513C00362000,79.77,80.17,0,QQQ 240513P00362000,0.00,0.01,0
4,QQQ20240513363,442.06,2024-05-13,2,363.0,79.06,0.178845,QQQ 240513C00363000,78.77,79.17,0,QQQ 240513P00363000,0.00,0.01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4143,QQQ20261218650,442.06,2026-12-18,951,650.0,207.94,0.470389,QQQ 261218C00650000,10.50,13.50,0,QQQ 261218P00650000,205.57,210.50,0
4144,QQQ20261218655,442.06,2026-12-18,951,655.0,212.94,0.481699,QQQ 261218C00655000,10.00,14.50,0,QQQ 261218P00655000,210.57,215.50,0
4145,QQQ20261218660,442.06,2026-12-18,951,660.0,217.94,0.493010,QQQ 261218C00660000,9.04,14.00,0,QQQ 261218P00660000,215.57,220.50,0
4146,QQQ20261218665,442.06,2026-12-18,951,665.0,222.94,0.504321,QQQ 261218C00665000,8.54,13.32,0,QQQ 261218P00665000,220.57,225.50,0


In [5]:
from invaas.schwab.cnn_fear_greed_index import get_historical_cnn_fear_greed_index


def get_fear_greed_index_data():
    historical_period = 14
    df = pd.DataFrame(data=get_historical_cnn_fear_greed_index()["data"])
    df.set_index(
        pd.DatetimeIndex([pd.Timestamp(x, unit="ms", tz="UTC") for x in df.x]),
        inplace=True,
    )
    df.rename(columns={"y": "fear_greed_index"}, inplace=True)
    df.sort_index(inplace=True)
    df["fear_greed_index"] = df.fear_greed_index.fillna(method="ffill").astype(float)
    df["previous_max_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).max()
    )
    df["previous_min_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).min()
    )
    return (
        int(df.fear_greed_index.values[-1]),
        int(df.previous_max_fear_greed_index.values[-1]),
        int(df.previous_min_fear_greed_index.values[-1]),
    )


current_fear_greed_index, previous_max_fear_greed_index, previous_min_fear_greed_index = get_fear_greed_index_data()

print(f"Current fear greed index: {current_fear_greed_index}")
print(f"Previous max fear greed index: {previous_max_fear_greed_index}")
print(f"Previous min fear greed index: {previous_min_fear_greed_index}")
print()

good_call_buy = current_fear_greed_index >= previous_max_fear_greed_index - 2
good_put_buy = current_fear_greed_index <= previous_min_fear_greed_index + 2

print(f"Buy call: {good_call_buy}")
print(f"Buy put: {good_put_buy}")

Current fear greed index: 47
Previous max fear greed index: 47
Previous min fear greed index: 33

Buy call: True
Buy put: False


In [6]:
min_dte = 14
min_volume = 100
max_strike_distance_pct = 0.1
max_buy_price = 100

df_options_to_buy = df_options_chains.loc[
    (df_options_chains.DTE > min_dte)
    & (df_options_chains.STRIKE_DISTANCE_PCT < max_strike_distance_pct)
    & (
        (good_call_buy & (df_options_chains.C_VOLUME > min_volume) & (df_options_chains.C_ASK * 100 < max_buy_price))
        | (good_put_buy & (df_options_chains.P_VOLUME > min_volume) & (df_options_chains.P_ASK * 100 < max_buy_price))
    )
].sort_values(by="STRIKE_DISTANCE_PCT")
df_options_to_buy

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
1503,QQQ20240531460,442.06,2024-05-31,20,460.00,17.94,0.040583,QQQ 240531C00460000,0.90,0.91,304,QQQ 240531P00460000,18.02,18.45,8
1504,QQQ20240531465,442.06,2024-05-31,20,465.00,22.94,0.051893,QQQ 240531C00465000,0.40,0.42,235,QQQ 240531P00465000,22.88,23.30,1
1613,QQQ20240607465,442.06,2024-06-07,27,465.00,22.94,0.051893,QQQ 240607C00465000,0.87,0.89,129,QQQ 240607P00465000,22.96,23.27,2
1614,QQQ20240607470,442.06,2024-06-07,27,470.00,27.94,0.063204,QQQ 240607C00470000,0.44,0.46,253,QQQ 240607P00470000,27.91,28.23,4
1907,QQQ20240621473,442.06,2024-06-21,41,473.00,30.94,0.069990,QQQ 240621C00473000,0.90,0.92,403,QQQ 240621P00473000,30.93,31.21,0
1725,QQQ20240614475,442.06,2024-06-14,34,475.00,32.94,0.074515,QQQ 240614C00475000,0.48,0.51,131,QQQ 240614P00475000,32.93,33.21,0
1910,QQQ20240621475,442.06,2024-06-21,41,475.00,32.94,0.074515,QQQ 240621C00475000,0.73,0.74,479,QQQ 240621P00475000,32.93,33.21,0
1913,QQQ20240621478,442.06,2024-06-21,41,478.00,35.94,0.081301,QQQ 240621C00478000,0.53,0.54,105,QQQ 240621P00478000,35.92,36.21,0
1915,QQQ20240621479.78,442.06,2024-06-21,41,479.78,37.72,0.085328,QQQ 240621C00479780,0.43,0.45,110,QQQ 240621P00479780,37.70,37.99,0
1616,QQQ20240607480,442.06,2024-06-07,27,480.00,37.94,0.085825,QQQ 240607C00480000,0.11,0.13,161,QQQ 240607P00480000,37.91,38.23,0


In [7]:
positions = task.schwab_api.get_balance_positions()
owned_option_positions = [
    x for x in positions["positionDetails"]["positions"] if x["securityType"] == "Option" and x["shares"] > 0
]
owned_call_options = [x for x in owned_option_positions if x["symbolDescription"].startswith("CALL")]
owned_put_options = [x for x in owned_option_positions if x["symbolDescription"].startswith("PUT")]
print(owned_call_options)
print(owned_put_options)
positions

KeyError: 'positions'

In [ ]:
df_owned_options_chains = df_options_chains.loc[
    (df_options_chains.C_ID.isin(owned_call_options)) | (df_options_chains.P_ID.isin(owned_put_options))
]
df_owned_options_chains

In [ ]:
task.schwab_api.get_balance_positions()
# task.schwab_api.trade(
#     ticker=df_owned_options_chains.C_ID.values[0], asset_class="Option", side="Sell", qty=1, dry_run=False
# )

In [ ]:
task.schwab_api.get_transaction_history()